In [16]:
#see readme for setup instructions of the chrome web driver

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse, parse_qs
# Initialize the browser instance
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run Chrome in headless mode
driver = webdriver.Chrome(options=options)

In [18]:
def main():
    sign_in("fid_1", "fid_1")
    agree_to_terms()
    select_all_accounts_and_approve()
    code = wait_for_404_page_and_get_code()
    print("=========== completed retrieving code =============")
    print(code)

def sign_in(username, password):
    print("signing in...")
    driver.get("https://sandbox-idp.ddp.akoya.com/auth?connector=mikomo&client_id=vpvuomqv25pmesud3n6nfd23a&redirect_uri=https%3A%2F%2Frecipient.dev.ddp.akoya.com%2Fflow%2Fcallback&response_type=code&scope=openid%20email%20profile%20offline_access&state=qwe4263gfa3")

    print("waiting for login redirect...")
    # Set up a wait object with a timeout (e.g., 10 seconds)
    wait = WebDriverWait(driver, 10)
    element = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[class^='LoginContent']"))
    )
    print(f"logincontent :{element}")
    inputs = driver.find_elements(By.TAG_NAME, "input")
    username_input = inputs[0]
    password_input = inputs[1]

    username_input.send_keys(username)
    password_input.send_keys(password)
    print("submitting login form...")
    password_input.send_keys(Keys.RETURN)

def agree_to_terms():
    wait = WebDriverWait(driver, 10)
    print("waiting for Next button to appear...")
    button_element = wait.until(
        EC.presence_of_element_located((By.XPATH, "//button[@aria-label='Next']"))
    )
    print("clicking Next button...")
    button_element.click()

def select_all_accounts_and_approve():
    wait = WebDriverWait(driver, 10)
    approve_button_element = wait.until(
        EC.presence_of_element_located((By.XPATH, "//button[@aria-label='Approve']"))
    )

    account_checkboxes = driver.find_elements(By.CSS_SELECTOR, 'input[type="checkbox"]')
    print(f"found {len(account_checkboxes)} account checkboxes")

    print("selecting all checkboxes")
    for account_checkbox in account_checkboxes:
        if not account_checkbox.is_selected():
            # account_checkbox.click() <- throws an error.  need to click label
            checkbox_id = account_checkbox.get_attribute('id')
            label_element = driver.find_element(By.CSS_SELECTOR, f'label[for="{checkbox_id}"]')
            label_element.click()
            print(f"checkbox_id: {checkbox_id} is now clicked: {account_checkbox.is_selected()}")

    print("clicking Approve button...")
    approve_button_element.click()

def wait_for_404_page_and_get_code():
    wait = WebDriverWait(driver, 10)
    print("waiting to be redirected to 404 page")
    sorry_element = wait.until(
        EC.text_to_be_present_in_element((By.TAG_NAME, 'h6'), "Sorry, we can't find this page.")
    )
    print("sorry text found.  parsing code from url query string param...")
    url = driver.current_url
    code = parse_code_from_url_youre_redirected_to_after_linking(url)
    # print(f"code parsed: {code}")
    return code

def parse_code_from_url_youre_redirected_to_after_linking(url):
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
    code = query_params.get("code", [None])[0]
    return code

main()

signing in...
waiting for login redirect...
logincontent :<selenium.webdriver.remote.webelement.WebElement (session="ad839fa2065579bba2265da497860e68", element="5B50EB4CC641B2A4595F224919DF1417_element_7")>
submitting login form...
waiting for Next button to appear...
clicking Next button...
found 16 account checkboxes
selecting all checkboxes
checkbox_id: 574602170:::401K is now clicked: True
checkbox_id: 5796964718:::Employee Stock Purchase is now clicked: True
checkbox_id: 1664171250:::HSA is now clicked: True
checkbox_id: 7762294619:::I is now clicked: True
checkbox_id: 5176476528:::I3 is now clicked: True
checkbox_id: 3591927705:::I3 is now clicked: True
checkbox_id: 6060055889:::IRA is now clicked: True
checkbox_id: 8478960276:::IRA is now clicked: True
checkbox_id: 830915738:::IRRL is now clicked: True
checkbox_id: 3737588790:::IS is now clicked: True
checkbox_id: 7314630770:::J is now clicked: True
checkbox_id: 116151841:::Performance Awards is now clicked: True
checkbox_id: 16